# ICoM Simulation for XGBOOST notebook

For this running the simulation please make sure you have:

1)list of csv with countries
2)csv contating geographic data

In this notebook we will simulate the ICoM Movment by predicting the growth in each country and cacluate the predicted cases in each country.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from openpyxl import load_workbook
from sklearn.model_selection import train_test_split
import xgboost as xgb
import shap

pd.options.mode.chained_assignment = None  

In [2]:
#train function

def train(df):
    x=df.iloc[10:32,17:-1]
    y=df.iloc[10:32,-1]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1,random_state=0)


    regressor = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1,  subsample=0.5,
                           colsample_bytree=0.1, max_depth=3,random_state=0,objective="reg:squarederror",
                        min_child_weight=1
                          )
    
    regressor.fit(x_train,y_train)
    feature_importances = pd.DataFrame(regressor.feature_importances_,
                                   index = x_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)

    #print(feature_importances)
    return regressor,feature_importances

In [3]:
#prediction function on t+1 of weekly growth in each country

def predict(df,i,regressor):
    
    x=df.iloc[:33+i,17:-1]
    y=df.iloc[:33+i,-1]
    predictions=regressor.predict(x)
    
    """
    df = pd.DataFrame({'Actual_rf': y, 'Predicted_rf': predictions})
    df.plot(kind='bar',figsize=(10,10))
    plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
    plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
    plt.title("Random forest",fontsize=16)
    plt.xlabel('Date ',fontsize=16)
    plt.ylabel('Growth',fontsize=16)
    plt.show()
    """
    
    #print ("Xgboost Model Score: %s" % (regressor.score(x, y)))
    
    return predictions

In [4]:
#create ICom based on the caculated confirmed cases based on the prediction of the weekly growth

def ICoM(gdict, tseries, c_r, states):

    icom = []
    numerator = []
    denominator = []
    for state in states:
        coordinates = np.array([gdict[state][0], gdict[state][1]])
        numerator.append(c_r[state] * coordinates)
        denominator.append(c_r[state])
    numerator = np.sum(np.array(numerator), axis=0)
    denominator = sum(denominator)
    if denominator == 0:
        icom.append(np.array([0, 0]))
    else:
        icom.append(numerator * (1/denominator))
    icom = np.array(icom)
    icom = pd.DataFrame({'date': tseries, 'lat': icom[:, 0], 'long': icom[:, 1]})
    dist=distFromCenter(icom,gdict,states)
    return icom , dist

In [5]:
#create distance from the center based on the ICoM that was created
def distFromCenter(centers, gdict, states):

    weekly_dist = {}
    for index in centers.index:
        center = np.array(centers['lat'][index], centers['long'][index])
        for key in gdict:
            point = np.array(gdict[key])
            distance = np.linalg.norm(center - point)
            if key in states:
                if key not in weekly_dist.keys():
                    weekly_dist[key] = [distance]
                else:
                    weekly_dist[key].append(distance)
    return weekly_dist

In [6]:
#read geographic data
geo=pd.read_csv("geo.csv")
geo.rename(columns = {"Unnamed: 0":"state","0":"lat","1":"long"},inplace=True)
geo=geo.set_index('state').T.to_dict('list')

In [7]:
#create an empty dataframe to save the ICOM prediction
df_icom=pd.DataFrame()
last_val={}
df=pd.read_csv("France.csv")
t_s=df["date"][32:37]

In [8]:
#create a diconery of dataframes where key is the country name
state_dict={}
predictions={}
state_list=['Germany', 'Italy', 'Spain', 'Belgium', 'Switzerland', 'Austria',
'France','Finland', 'Greece', 'Netherlands']



for state in state_list:
    df=pd.read_csv(state+".csv")
    if state not in state_dict.keys():
        state_dict[state]=df

In [9]:
#create a copy of the original
original_data={}
for state in state_list:
    if state not in original_data.keys():
        original_data[state]=state_dict[state].copy()

In [10]:
#rename columns
for state in state_list:
    original_data[state]=original_data[state].rename(columns={"WeeklyConfirmed":"WeeklyConfirmed_real","WeeklyGrowth":"WeeklyGrowth_real",
                                           "Weeklydistance":"Weeklydistance_real"})

In [11]:
#create a dictionary of trained models and extrect feature imporance for each model
train_dict={}
feature_df = pd.DataFrame()
for state in state_list:
    if state not in train_dict.keys():
        train_dict[state],data=train(state_dict[state])
        
        
        feature_df=feature_df.append(data)
        #print(state)
    
        
        
feature_df.to_csv("feature importance XGBOOST.csv")

In [12]:
"""
Create prediction for the Weekly Growth for every trained model 
Caculate the Weekly Confiremd Cases based on the predicted Growth for t+1
Caculate New ICoM Based on the predicted Confirmed Cases
Caculate distance from Center of ICoM for each country
"""

for i in range(len(t_s)):

   
    for state in state_list:
        
        if state not in predictions.keys():
            
            predictions[state]=predict(state_dict[state],i,train_dict[state])
            
            
        state_dict[state]["WeeklyGrowth"][32+i]=predictions[state][32+i]
        state_dict[state]["WeeklyConfirmed"][32+i]=state_dict[state]["WeeklyGrowth"][32+i]*state_dict[state]["WeeklyConfirmed"][31+i]
        
        if state not in last_val.keys():
            last_val[state]= state_dict[state]["WeeklyConfirmed"][32+i]
            

    
    icom ,dist =ICoM(geo,i,last_val,state_list)
    df_icom=df_icom.append(icom)
    for state in state_list:
        
        state_dict[state]["Weeklydistance"][32+i]=dist[state][0]
        
    
    last_val.clear()
    predictions.clear()
    print(icom)
    print("\n")
        
    
    
    

    

   date        lat      long
0     0  45.027635  4.142377


   date        lat      long
0     1  44.940221  3.896906


   date        lat      long
0     2  44.850458  3.655784


   date        lat      long
0     3  44.760259  3.419197


   date        lat      long
0     4  44.668229  3.189073




# Write all the Data to excel

In [13]:
import xlsxwriter


workbook = xlsxwriter.Workbook('predictions_growth_xgboost.xlsx')
workbook.close()

In [14]:
book = load_workbook('predictions_growth_xgboost.xlsx')
writer = pd.ExcelWriter('predictions_growth_xgboost.xlsx', engine = 'openpyxl')
writer.book = book
for state in state_list:
    

    
    df["date"].to_excel(writer, sheet_name=state,index=False)
    state_dict[state]["WeeklyConfirmed"].to_excel(writer, sheet_name=state,index=False,startcol=1)
    state_dict[state]["WeeklyGrowth"].to_excel(writer, sheet_name=state,index=False,startcol=2)
    state_dict[state]["Weeklydistance"].to_excel(writer, sheet_name=state,index=False,startcol=3)
    original_data[state]["WeeklyConfirmed_real"].to_excel(writer, sheet_name=state,startcol=4,index=False)
    original_data[state]["WeeklyGrowth_real"].to_excel(writer, sheet_name=state,startcol=5,index=False)
    original_data[state]["Weeklydistance_real"].to_excel(writer, sheet_name=state,startcol=6,index=False)
    writer.save()
writer.close()
    

In [15]:
book = load_workbook('predictions_growth_xgboost.xlsx')
writer = pd.ExcelWriter('predictions_growth_xgboost.xlsx', engine = 'openpyxl')
writer.book = book
df_icom.to_excel(writer,sheet_name="ICOM")
writer.save()
writer.close()